In [2]:
import PyPDF2

In [3]:
import os
from os import listdir
from os.path import isfile, join

from io import StringIO

In [4]:
import pandas as pd

from collections import Counter

In [5]:
import en_core_web_sm

In [6]:
nlp = en_core_web_sm.load()

In [7]:
from spacy.matcher import PhraseMatcher

In [18]:
mypath = "C:\\Users\\LUY1\\Desktop\\practice\\Resume\\CVs"

In [19]:
onlyfiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [1]:
#onlyfiles

In [21]:
# function to read resumes

def pdfextract(file):
    fileReader = PyPDF2.PdfFileReader(open(file, 'rb'))
    countpage = fileReader.getNumPages()
    count = 0
    text = []
    while count < countpage:
        pageObj = fileReader.getPage(count)
        count += 1
        t = pageObj.extractText()
        print(t)
        text.append(t)
    return text

In [59]:
# function to do phrase matching and build candidate profile

def create_profile(file):
    text = pdfextract(file)
    text = str(text)
    text = text.replace("\\n", "")
    text = text.lower()
    
    keyword_dict = pd.read_csv("C:\\Users\\LUY1\\Desktop\\practice\\Resume\\template_new.csv", encoding = "ISO-8859-1")
    stats_words = [nlp(text) for text in keyword_dict['Statistics'].dropna(axis = 0)]
    NLP_words = [nlp(text) for text in keyword_dict['NLP'].dropna(axis = 0)]
    ML_words = [nlp(text) for text in keyword_dict['Machine Learning'].dropna(axis = 0)]
    DL_words = [nlp(text) for text in keyword_dict['Deep Learning'].dropna(axis = 0)]
    R_words = [nlp(text) for text in keyword_dict['R Language'].dropna(axis = 0)]
    python_words = [nlp(text) for text in keyword_dict['Python Language'].dropna(axis = 0)]
    Data_Engineering_words = [nlp(text) for text in keyword_dict['Data Engineering'].dropna(axis = 0)]
    
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('Stats', None, *stats_words)
    matcher.add('NLP', None, *NLP_words)
    matcher.add('ML', None, *ML_words)
    matcher.add('DL', None, *DL_words)
    matcher.add('R', None, *R_words)
    matcher.add('Python', None, *R_words)
    matcher.add('DE', None, *Data_Engineering_words)
    doc = nlp(text)
    
    d = []
    matches = matcher(doc)
    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        d.append((rule_id, span.text))
    keywords = "\n".join(f"{i[0]} {i[1]} ({j})" for i, j in Counter(d).items())
    
    #converting string of keywords to dataframe
    df = pd.read_csv(StringIO(keywords), names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ', 1).tolist(), columns = ['Subject', 'Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(', 1).tolist(), columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'], df2['Keyword'], df2['Count']], axis = 1)
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    
    base = os.path.basename(file)
    filename = os.path.splitext(base)[0]
    
    name = filename.split('_')
    name2 = name[0]
    name2 = name2.lower()
    
    #converting str to dataframe
    name3 = pd.read_csv(StringIO(name2), names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)
    
    return(dataf)    

In [52]:
final_database = pd.DataFrame()

In [2]:
# i = 0

# while i < len(onlyfiles):
#     file = onlyfiles[i]
#     dat = create_profile(file)
#     final_database = final_database.append(dat)
#     i += 1
#     print(final_database)

In [3]:
# file = onlyfiles[2]
# print(file)

In [4]:
# dat = create_profile(file)
# dat

In [5]:
# text = pdfextract(file)
# text

In [36]:
keyword_dict = pd.read_csv("C:\\Users\\LUY1\\Desktop\\practice\\Resume\\template_new.csv",
                          encoding = "ISO-8859-1")
keyword_dict

,Statistics,Machine Learning,Deep Learning,R Language,Python Language,NLP,Data Engineering
0,statistical models,linear regression,keras,ggplot,flask,nlp,aws
1,statisticalmodeling,logistic regression,theano,shiny,django,natural language processing,ec2
2,probability,K means,face detection,cran,pandas,topic modeling,amazon redshift
3,normal distribution,random forest,neural networks,dplyr,numpy,lda,s3
4,poisson distribution,xgboost,convolutional neural network (cnn),tidyr,scikitlearn,named entity recognition,docker
5,survival models,Random forest,recureent neural network (RNN),lubridate,sklearn,pos taging,kubernets
6,hypothesis testing,naïve bayes,object detction,knitr,matplotlib,word2vec,scala
7,factor analysis,pca,yolo,NaN,scipy,word embedding,teradata
8,forecasting,decision trees,gpu,NaN,bokeh,lsi,google big query
9,markov chain,svd,cuda,NaN,statsmodel,spacy,aws emr
